# Azure ML Classification Model Inference

This notebook demonstrates how to call the deployed Azure ML classification model endpoint for traffic citation predictions.

## Setup

Import required libraries and set up the endpoint configuration.

In [ ]:
import urllib.request
import json
import pandas as pd
from datetime import datetime

## Configuration

Set the endpoint URL and API key. **Replace the empty API key with your actual key.**

In [ ]:
# Endpoint configuration
url = 'https://tech901-workspace-vzkrc.eastus2.inference.ml.azure.com/score'

# TODO: Replace with your actual API key
api_key = 'API KEY HERE'

if not api_key or api_key == 'YOUR_API_KEY_HERE':
    raise Exception("A key should be provided to invoke the endpoint")

## Prepare Sample Data

Create sample traffic citation data that matches the model's expected input schema:
- Ticket Number (string)
- Ticket Issue Date (datetime)
- Citation Type (string)
- Ordinance Violated (string)
- Vehicle Year (float)
- Vehicle Make (string)
- Vehicle Model (string)
- Driver Gender (string)
- Driver Age (integer)
- Officer IBM (float)
- Officer Name (string)
- Location (string)
- Driver License ZIP Code (float)
- ObjectId (integer)

In [ ]:
# Sample data for inference
sample_data = {
    "input_data": {
        "columns": [
            "Ticket Number",
            "Ticket Issue Date",
            "Citation Type",
            "Ordinance Violated",
            "Vehicle Year",
            "Vehicle Make",
            "Vehicle Model",
            "Driver Gender",
            "Driver Age",
            "Officer IBM",
            "Officer Name",
            "Location",
            "Driver License ZIP Code",
            "ObjectId"
        ],
        "data": [
            [
                "T123456",
                "2024-01-15T10:30:00Z",
                "Traffic",
                "55-8-152",
                2018.0,
                "Toyota",
                "Camry",
                "M",
                35,
                12345.0,
                "Officer Smith",
                "Main St & 1st Ave",
                38103.0,
                1001
            ],
            [
                "T123457",
                "2024-01-15T14:45:00Z",
                "Parking",
                "55-8-160",
                2020.0,
                "Honda",
                "Accord",
                "F",
                28,
                12346.0,
                "Officer Johnson",
                "Poplar Ave & Highland",
                38111.0,
                1002
            ],
            [
                "T123458",
                "2024-01-16T09:15:00Z",
                "Traffic",
                "55-8-104",
                2015.0,
                "Ford",
                "F-150",
                "M",
                42,
                12347.0,
                "Officer Williams",
                "Union Ave & McLean",
                38104.0,
                1003
            ]
        ]
    }
}

# Display the sample data
print("Sample data prepared:")
print(json.dumps(sample_data, indent=2))

df = pd.DataFrame(

## Make Prediction Request

Send the data to the endpoint and get predictions.

In [ ]:
# Prepare the request
body = str.encode(json.dumps(sample_data))

headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': ('Bearer ' + api_key)
}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    
    print("Prediction successful!")
    print("\nRaw response:")
    print(result.decode('utf-8'))
    
    # Parse and display results
    predictions = json.loads(result)
    print("\nParsed predictions:")
    print(json.dumps(predictions, indent=2))
    
except urllib.error.HTTPError as error:
    print(f"The request failed with status code: {error.code}")
    print("\nError details:")
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

## Analyze Results

Display the predictions in a more readable format.

In [ ]:
# Display predictions with corresponding input data
if 'predictions' in locals():
    print("\nPrediction Results Summary:")
    print("=" * 80)
    
    for i, (record, prediction) in enumerate(zip(sample_data['input_data']['data'], predictions)):
        print(f"\nRecord {i+1}:")
        print(f"  Ticket Number: {record[0]}")
        print(f"  Vehicle: {record[6]} {record[5]} ({record[4]})")
        print(f"  Driver: {record[7]}, Age {record[8]}")
        print(f"  Location: {record[11]}")
        print(f"  Prediction: {prediction}")
        print("-" * 80)
else:
    print("No predictions available. Please run the prediction request cell first.")